In [1]:
import pandas as pd
import psycopg2

In [2]:
def create_database():
    # connect to default database
    conn = psycopg2.connect(host='localhost', dbname='postgres', user='postgres', password='root')
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    
    # create sparkify database with UTF8 encoding
    cur.execute('DROP DATABASE IF EXISTS olist')
    cur.execute('CREATE DATABASE olist')
    
    # close connection to default database
    conn.close()
    
    # connect to sparkify database
    conn = psycopg2.connect(host='localhost', dbname='olist', user='postgres', password='root')
    cur = conn.cursor()
    
    return cur, conn

In [3]:
# Reading in data files (9 CSV's)

In [4]:
olist_customers = pd.read_csv('olist_brazil_ecommerce_data/olist_customers_dataset.csv')
olist_customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [5]:
# customer_id is actually the key order_id, and customer_unique_id is the key for customer_id, so we will rename these columns
olist_customers.rename(columns = {'customer_id':'order_id', 'customer_unique_id':'customer_id'}, inplace=True)
olist_customers.head()

,order_id,customer_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [6]:
olist_order_items = pd.read_csv('olist_brazil_ecommerce_data/olist_order_items_dataset.csv')
# Reordering columns to fit schema
olist_order_items = olist_order_items[['order_item_id', 'order_id', 'product_id', 'seller_id', 'shipping_limit_date', 'price', 'freight_value']]
olist_order_items.head()

,order_item_id,order_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,1,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,1,00018f77f2f0320c557190d7a144bdd3,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,1,000229ec398224ef6ca0657da4fc703e,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,1,00024acbcdf0a6daa1e931b038114c75,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,1,00042b26cf59d7ce69dfabb4e55b4fd9,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [7]:
olist_orders = pd.read_csv('olist_brazil_ecommerce_data/olist_orders_dataset.csv')
# Filling in NaN values with '1001-01-01 00:00:00', currently looking for a way to fill will null value instead of providing a "default" timestamp
olist_orders[['order_approved_at','order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']] = olist_orders[['order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']].fillna('1001-01-01 00:00:00')
olist_orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [8]:
olist_order_payments = pd.read_csv('olist_brazil_ecommerce_data/olist_order_payments_dataset.csv')
olist_order_payments.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [9]:
olist_order_reviews = pd.read_csv('olist_brazil_ecommerce_data/olist_order_reviews_dataset.csv')
olist_order_reviews.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [10]:
olist_products = pd.read_csv('olist_brazil_ecommerce_data/olist_products_dataset.csv')
# Filling in NaN / Empty values with 0
olist_products[['product_name_length', 'product_description_length', 'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']] = olist_products[['product_name_length', 'product_description_length', 'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']].fillna(0)
# Casting columns as int, was being spit out as a float
olist_products[['product_name_length', 'product_description_length', 'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']] = olist_products[['product_name_length', 'product_description_length', 'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']].astype(int)
olist_products.head()

,product_id,product_category_name,product_name_length,product_description_length,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40,287,1,225,16,10,14
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44,276,1,1000,30,18,20
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46,250,1,154,18,9,15
3,cef67bcfe19066a932b7673e239eb23d,bebes,27,261,1,371,26,4,26
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37,402,4,625,20,17,13


In [11]:
olist_sellers = pd.read_csv('olist_brazil_ecommerce_data/olist_sellers_dataset.csv')
olist_sellers.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


In [12]:
olist_category_name_translation = pd.read_csv('olist_brazil_ecommerce_data/product_category_name_translation.csv')
olist_category_name_translation.head()

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


In [13]:
olist_geolocation = pd.read_csv('olist_brazil_ecommerce_data/olist_geolocation_dataset.csv')
olist_geolocation.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


In [14]:
cur, conn = create_database()

In [15]:
# Database table creation queries

In [16]:
customer_table_create = ("""CREATE TABLE IF NOT EXISTS customers(
                         customer_id VARCHAR PRIMARY KEY,
                         order_id VARCHAR,
                         customer_zip_code INT,
                         customer_city VARCHAR,
                         customer_state VARCHAR
                         )""")
cur.execute(customer_table_create)
conn.commit()

In [17]:
order_items_table_create = ("""CREATE TABLE IF NOT EXISTS order_items(
                            order_item_id INT,
                            order_id VARCHAR,
                            product_id VARCHAR,
                            seller_id VARCHAR,
                            shipping_limit_date TIMESTAMP WITHOUT TIME ZONE,
                            price FLOAT,
                            freight_value FLOAT
                            )""")
cur.execute(order_items_table_create)
conn.commit()

In [18]:
orders_table_create = ("""CREATE TABLE IF NOT EXISTS orders(
                        order_id VARCHAR PRIMARY KEY,
                        customer_id VARCHAR,
                        order_status VARCHAR,
                        purchase_timestamp TIMESTAMP WITHOUT TIME ZONE,
                        order_approved_at TIMESTAMP WITHOUT TIME ZONE,
                        carrier_delivered_date TIMESTAMP WITHOUT TIME ZONE,
                        customer_delivered_date TIMESTAMP WITHOUT TIME ZONE,
                        estimated_delivery_date TIMESTAMP WITHOUT TIME ZONE
                        )""")
cur.execute(orders_table_create)
conn.commit()

In [19]:
order_payments_table_create = ("""CREATE TABLE IF NOT EXISTS order_payments(
                                order_id VARCHAR,
                                payment_sequential INT,
                                payment_type VARCHAR,
                                payment_installments VARCHAR,
                                payment_value FLOAT
                                )""")
cur.execute(order_payments_table_create)
conn.commit()

In [20]:
order_reviews_table_create = ("""CREATE TABLE IF NOT EXISTS order_reviews(
                                review_id VARCHAR,
                                order_id VARCHAR,
                                review_score INT,
                                review_comment_title VARCHAR,
                                review_comment_message VARCHAR,
                                review_creation_date TIMESTAMP WITHOUT TIME ZONE,
                                review_answer_timestamp TIMESTAMP WITHOUT TIME ZONE
                                )""")
cur.execute(order_reviews_table_create)
conn.commit()

In [21]:
products_table_create = ("""CREATE TABLE IF NOT EXISTS products(
                        product_id VARCHAR PRIMARY KEY,
                        product_category VARCHAR,
                        product_name_length INT,
                        product_desc_length INT,
                        num_photos INT,
                        weight_in_grams INT,
                        product_len_cm INT,
                        product_height_cm INT,
                        product_width_cm INT
                        )""")
cur.execute(products_table_create)
conn.commit()

In [22]:
sellers_table_create = ("""CREATE TABLE IF NOT EXISTS sellers(
                        seller_id VARCHAR PRIMARY KEY,
                        seller_zip_code INT,
                        seller_city VARCHAR,
                        seller_state VARCHAR
                        )""")
cur.execute(sellers_table_create)
conn.commit()

In [23]:
category_name_table_create = ("""CREATE TABLE IF NOT EXISTS category_name_translation(
                                product_category VARCHAR PRIMARY KEY,
                                product_category_en VARCHAR
                                )""")
cur.execute(category_name_table_create)
conn.commit()

In [24]:
geolocation_table_create = ("""CREATE TABLE IF NOT EXISTS geolocation(
                            zip_code INT,
                            latitude FLOAT,
                            longitude FLOAT,
                            geolocation_city VARCHAR,
                            geolocation_state VARCHAR
                            )""")
cur.execute(geolocation_table_create)
conn.commit()

In [25]:
# Database table insertion queries

In [26]:
customers_table_insert = ("""INSERT INTO customers(
                        customer_id,
                        order_id,
                        customer_zip_code,
                        customer_city,
                        customer_state)
                        VALUES (%s, %s, %s, %s, %s)
                        """)

In [27]:
for i, row in olist_customers.iterrows():
    cur.execute(customers_table_insert, list(row))

In [28]:
order_items_table_insert = ("""INSERT INTO order_items(
                            order_item_id,
                            order_id,
                            product_id,
                            seller_id,
                            shipping_limit_date,
                            price,
                            freight_value)
                            VALUES (%s, %s, %s, %s, %s, %s, %s)
                            """)

In [29]:
for i, row in olist_order_items.iterrows():
    cur.execute(order_items_table_insert, list(row))

In [30]:
orders_table_insert = ("""INSERT INTO orders(
                    order_id,
                    customer_id,
                    order_status,
                    purchase_timestamp,
                    order_approved_at,
                    carrier_delivered_date,
                    customer_delivered_date,
                    estimated_delivery_date)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                    """)

In [31]:
for i, row in olist_orders.iterrows():
    cur.execute(orders_table_insert, list(row))

In [32]:
order_payments_table_insert = ("""INSERT INTO order_payments(
                                order_id,
                                payment_sequential,
                                payment_type,
                                payment_installments,
                                payment_value)
                                VALUES (%s, %s, %s, %s, %s)
                                """)

In [33]:
for i, row in olist_order_payments.iterrows():
    cur.execute(order_payments_table_insert, list(row))

In [34]:
order_reviews_table_insert = ("""INSERT INTO order_reviews(
                                review_id,
                                order_id,
                                review_score,
                                review_comment_title,
                                review_comment_message,
                                review_creation_date,
                                review_answer_timestamp)
                                VALUES (%s, %s, %s, %s, %s, %s, %s)
                                """)

In [35]:
for i, row in olist_order_reviews.iterrows():
    cur.execute(order_reviews_table_insert, list(row))

In [36]:
products_table_insert = ("""INSERT INTO products(
                        product_id,
                        product_category,
                        product_name_length,
                        product_desc_length,
                        num_photos,
                        weight_in_grams,
                        product_len_cm,
                        product_height_cm,
                        product_width_cm)
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
                        """)

In [37]:
for i, row in olist_products.iterrows():
    cur.execute(products_table_insert, list(row))

In [38]:
sellers_table_insert = ("""INSERT INTO sellers(
                        seller_id,
                        seller_zip_code,
                        seller_city,
                        seller_state)
                        VALUES (%s, %s, %s, %s)
                        """)

In [39]:
for i, row in olist_sellers.iterrows():
    cur.execute(sellers_table_insert, list(row))

In [40]:
category_name_table_insert = ("""INSERT INTO category_name_translation(
                                product_category,
                                product_category_en)
                                VALUES (%s, %s)
                                """)

In [41]:
for i, row in olist_category_name_translation.iterrows():
    cur.execute(category_name_table_insert, list(row))

In [42]:
geolocation_table_insert = ("""INSERT INTO geolocation(
                            zip_code,
                            latitude,
                            longitude,
                            geolocation_city,
                            geolocation_state)
                            VALUES (%s, %s, %s, %s, %s)
                            """)

In [43]:
for i, row in olist_geolocation.iterrows():
    cur.execute(geolocation_table_insert, list(row))

In [44]:
conn.commit()